# Tests unsuitable for unittest

Some of the functions included in the `gilbert_elliot_model` package are not well suited to be covered by `unittest` tests as they are both slow and involve many random processes. The randomness requires running many repetitions and considering the distribution of results, but the slowness causes this to be too inefficient to do in the `unittest` format. Thus this document analyzes results that were obtained via `distribution_tests.py` so that we can consider the distributions of results to characterize how parts of the package are working.

The code chunk below prepares us for analysis.

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

import gilbert_elliot_model as ge


## Fitting a Hidden Markov Model

The `gilbert_elliot_model` package is designed to take an observed error pattern and fit the model to it. The Gilbert-Elliot burst error model is a two-state hidden Markov model. Thus the package `hmmlearn` is primarily used for this fitting. However the function `gilbert_elliot_model.gilbert_elliot.fit_hmm` functions as a wrapper for `hmmlearn` functionality that is specifically tuned to the Gilbert-Elliot burst error model. 

The documentation for the function `gilbert_elliot_model.gilbert_elliot.fit_hmm` is provided below.

In [ ]:
help(ge.gilbert_elliot.fit_hmm)

In [ ]:
fixed_param_fname = 'Fixed_Parameter_README.md'
fixed_param_vals = pd.read_csv(fixed_param_fname, skiprows=1)

In [ ]:
def check_k_h(df, stats, fixed_param_vals):
    for stat in stats:
        stat_err = stat + '_error'
        if stat_err not in df.columns:
            if stat == 'k':
                df[stat_err] = df[stat] - 1
            elif stat == 'h':
                 df[stat_err] = df[stat]  
        print(f'All values of {stat} are correct: {all(df[stat_err] == 0)}')
def error_histograms(df, stats, fixed_param_vals=None):
    for stat in stats:
        stat_err = stat + '_error'
        if stat_err not in df.columns:
            df[stat_err] = df[stat] - fixed_param_vals.loc[0, stat]

        stat_mean = np.mean(df[stat_err])
        stat_unc = np.std(df[stat_err])/np.sqrt(df.shape[0])
        stat_CI = stat_mean + np.array([-1 ,1]) * 1.96 * stat_unc

        fig = go.Figure()
        fig.add_trace(go.Histogram(x=df[stat_err]))
        fig.add_vline(x=0, line_dash='dot', line_color = 'red',
                     annotation_text='Success criteria', annotation_position='bottom left',
                     )
        fig.add_vline(x=stat_CI[0], line_dash='dash', line_color='black',)
        fig.add_vline(x=stat_CI[1], line_dash='dash', line_color='black',
                     annotation_text='95% Confidence Interval', annotation_position='top right'
                     )
        if fixed_param_vals is not None:
            title_str = f'{stat} error from expected ({fixed_param_vals.loc[0, stat]})'
        else:
            title_str = f'{stat} error'
        fig.update_layout(
            title={'text': title_str}
        )

        fig.update_xaxes(title='Error')
        fig.show(renderer='notebook')

def update_df_error_stats(df, expected_stats=None, stats=None):
    if expected_stats is None:
        est_flag = '_estimate'
        target_flag = '_target'
        expected_stats = {}
        for stat in stats:
            expected_stats[stat]= None
    else:
        est_flag = ''
    for out_stat, value in expected_stats.items():
        df[out_stat] = df.apply(
        lambda row : ge.model_error_statistics(p=row['p' + est_flag],
                                               r=row['r' + est_flag],
                                               k=row['k' + est_flag],
                                               h=row['h' + est_flag])[out_stat],axis=1,)
        expected_out_stat = 'expected_' + out_stat
        if value is None:
            df[expected_out_stat] = df.apply(
                lambda row : ge.model_error_statistics(p=row['p' + target_flag],
                                                       r=row['r' + target_flag],
                                                       k=row['k' + target_flag],
                                                       h=row['h' + target_flag])[out_stat],axis=1,)
        else:
            df[expected_out_stat] = value
        diff_out_stat = out_stat + '_error'
        df[diff_out_stat] = df[expected_out_stat] - df[out_stat]
    return df

## Methodology

Two types of tests were performed. One with fixed parameter values for 1000 repetitions and one with random parameter values for each repetition. For the two-parameter model, `k=1, h=0` were always fixed, for the three-parameter model `k=1` was always fixed. Otherwise, in fixed parameter tests, the parameters were set to the values below.



In [ ]:
print(fixed_param_vals)

For both test types we will consider distributions of model parameter estimations directly. However, as we are fitting a hidden Markov model we expect that there will be many combinations of model parameters that would emit similar error patterns. This means it is likely we will see discrepancies from expectation when directly considering model parameters. 

To alleviate this we will also consider the distributions of error statistics derived from model parameter estimates.

### Fixed values tests
In a fixed value test we fix some subset of `(p, r, k, h)` depending on which version of the Gilbert-Elliot model we are testing. We then repeat the following $N_{iter}$ times: 
* Generate error pattern from model parameters of length $N_{err}$.
* Estimate the corresponding parameters $(\hat{p}, \hat{r}, \hat{k}, \hat{h})$
* Calculate the error statistics from the estimates, e.g., $\hat{\bar{x}}(\hat{p}, \hat{r}, \hat{k}, \hat{h})$.

We can then compare the distribution of differences between error statistic estimates and expected error statistics. Here the expected error statistic is calculated from the true model parameters, e.g., $\bar{x}(p, r, k, h)$. For example we consider the distribution of error rate errors via $\bar{x} - \hat{\bar{x}}$.

The diagram below demonstrates this methodology.
![Fixed value test diagram](./figs/fixed_value_test_diagram.png)

### Random values tests
In a random value test the procedure is almost identical except we generate a new set of $(p, r, k, h)$ each iteration and also calculate the relevant true error statistics each iteration. This process is shown in the diagram below.
![Random value test diagram](./figs/random_value_test_diagram.png)

### Interpretation

We expect to see errors in the direct estimation of parameters for the three- and four- parameter models. This is because in these models we are dealing with a true hidden markov model, meaning the model parameters are not directly observable. However we expect that all error statistics that are directly observable, namely the error rate, $\bar{x}$ and expected burst length, $L_1$, will be well estimated and the observed errors in the model parameter estimations will be driven by error statstics we cannot directly observe such as the proportion of time spent in the bad state $\pi_B$.

## Results
For all these tests `N_iter=1000` and `N_err=1000`.

### Two-parameter fixed values

For the two-parameter fixed values test the distributions of both `p` and `r` match expectations. Upon inspection the 95% confidence interval for `p` barely does not contain 0, but the distribution is centered near 0 and the discrepancy is very small and does not indicate anything majorly wrong happening with the software.

In [ ]:
fname = 'two_param_distributions_fixed_values.csv'
df_2pf = pd.read_csv(fname)

# Confirm that all k values 1, all h values 0
check_k_h(df_2pf, stats=['k', 'h'], fixed_param_vals=fixed_param_vals)

stats = ['p', 'r']
error_histograms(df_2pf, stats=stats, fixed_param_vals=fixed_param_vals)

### Two-parameter random values

For the test where `p` and `r` are randomly generated each trial we see even better agreement and the distributions match expectations exactly.

In [ ]:
fname = 'two_param_distributions_random_errors.csv'
df_2pr = pd.read_csv(fname)
# Confirm that all k values 1, all h values 0
check_k_h(df_2pr, stats=['k', 'h'], fixed_param_vals=fixed_param_vals)

stats = ['p', 'r']
error_histograms(df_2pr, stats=stats,)

### Three-parameter fixed

When we move to the three-parameter case we can pretty quickly see that we do not get great agreement between the parameter estimates and truth. The histograms of errors for model parameters below shows this pretty well. As explained above, this doesn't mean that the fitting process is completely flawed, it seems natural that a variety of parameter pairs `(p, r, h)` could yield similar error pattern results and thus be difficult to distinguish. 

In [ ]:
fname = 'three_param_distributions_fixed_values.csv'
df_3pf = pd.read_csv(fname)
# Confirm that all k values 1, all h values 0
check_k_h(df_3pf, stats=['k'], fixed_param_vals=fixed_param_vals)

stats = ['p', 'r', 'h']
error_histograms(df_3pf, stats=stats, fixed_param_vals=fixed_param_vals)

In order to account for this discrepancy we instead consider the errors from the error statistics generated by the original model parameters compared to those computed by the estimate parameters. The plots below show that for the error statistics that are direclty observable, namely error rate and expected burst length, we estimate those well. However the proportion of time spent in the bad state is not directly observable and contributes to the erroneous estimates for `(p, r, h)`.

In [ ]:
expected_stats = ge.model_error_statistics(p=fixed_param_vals.loc[0, 'p'],
                                           r=fixed_param_vals.loc[0, 'r'],
                                           h=fixed_param_vals.loc[0, 'h'],
                                           k=1,
                                           )
df_3pf = update_df_error_stats(df_3pf, expected_stats=expected_stats)

In [ ]:
error_stats = ['error_rate', 'expected_burst_length', 'bad_proportion']
error_histograms(df_3pf, stats=error_stats)

### Aside
Note that for all the plots below, the analysis is essentially the same as above. Namely the software correctly identifies model parameters that generate the relevant error statistics but estimation errors occur when estimating non-directly-observable events/error statistics. These errors ultimately propagate into the model parameter estimates causing them to be off from truth.

### Three parameter-random

In [ ]:
fname = 'three_param_distributions_random_errors.csv'
df_3pr = pd.read_csv(fname)
# Confirm that all k values 1, all h values 0
check_k_h(df_3pr, stats=['k'], fixed_param_vals=fixed_param_vals)

stats = ['p', 'r', 'h']
error_histograms(df_3pr, stats=stats, )

In [ ]:
df_3pr = update_df_error_stats(df_3pr, stats=error_stats)

In [ ]:
error_histograms(df_3pr, stats=error_stats)

### Four parameter-fixed

In [ ]:
fname = 'four_param_distributions_fixed_values.csv'
df_4pf = pd.read_csv(fname)

stats = ['p', 'r', 'h', 'k']
error_histograms(df_4pf, stats=stats, fixed_param_vals=fixed_param_vals)

In [ ]:
expected_stats = ge.model_error_statistics(p=fixed_param_vals.loc[0, 'p'],
                                           r=fixed_param_vals.loc[0, 'r'],
                                           h=fixed_param_vals.loc[0, 'h'],
                                           k=fixed_param_vals.loc[0, 'k'],
                                           )
df_4pf = update_df_error_stats(df_4pf, expected_stats=expected_stats)

In [ ]:
error_histograms(df_4pf, stats=error_stats)

### Four-parameter random

In [ ]:
fname = 'four_param_distributions_random_errors.csv'
df_4pr = pd.read_csv(fname)

stats = ['p', 'r', 'h', 'k']
error_histograms(df_4pr, stats=stats, )

In [ ]:
df_4pr = update_df_error_stats(df_4pr, stats=error_stats)

In [ ]:
error_histograms(df_4pr, stats=error_stats)